In [34]:
import skimage
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
import numpy as np
import os
import scipy.ndimage.morphology
from skimage.measure import label, regionprops
from skimage import feature
from skimage import color
from skimage import filters
import cv2
from sklearn.cluster import KMeans, MiniBatchKMeans
import sklearn
import csv

# Load images

In [33]:
PATH_TO_RESOURCES='../data/images/'
file_paths = !ls $PATH_TO_RESOURCES*.jpg | sort
imgs = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY) for path in file_paths]

# Descriptors

In [18]:
desc = []
sift = cv2.xfeatures2d.SIFT_create()

for im in imgs:
    kp, d = sift.detectAndCompute(im, None)
    if d is not None:
        desc.append(d.astype(np.float32))
desc = np.asarray(desc)


In [19]:
train_desc = np.vstack(desc)
print(train_desc.shape)

(222,)


In [21]:
train_mean = np.mean(train_desc, axis=1, keepdims=True)
train_desc = train_desc - train_mean

In [22]:
train_cov = np.dot(train_desc.T, train_desc)
eigvals, eigvecs = np.linalg.eig(train_cov)
perm = eigvals.argsort()                   # sort by increasing eigenvalue
pca_transform = eigvecs[:, perm[64:128]]   # eigenvectors for the 64 last eigenvalues
pca_transform.shape, pca_transform.dtype

((128, 64), dtype('float32'))

In [23]:
train_desc = np.dot(train_desc, pca_transform)

In [24]:
train_desc.shape

(48664, 64)

# K-means

In [26]:
kmeans = MiniBatchKMeans(n_clusters=512, random_state=0).fit(train_desc)

/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


In [ ]:
kmeans.cluster_centers_.shape

# Split training set

In [81]:
all_images = []
labels = []
with open('../data/gt_img.csv', newline='') as csvfile:
    for line in csvfile:
        line = line.strip()
        im_name, y = line.split(',')
        all_images.append(im_name)
        labels.append(int(y))
len(all_images), len(labels)

(224, 224)

In [83]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(all_images,
                                                                          labels,
                                                                         test_size=0.2,
                                                                         random_state=42)
len(x_train), len(x_val), len(y_train), len(y_val)

(179, 45, 179, 45)

# Training

In [86]:
l2_normalizer = sklearn.preprocessing.Normalizer(norm='l2', copy=True)
l1_normalizer = sklearn.preprocessing.Normalizer(norm='l1', copy=True)

In [89]:
def compute_descriptors(filelist, train_mean, pca_transform, kmeans):
    image_descriptors = []
    sift = cv2.xfeatures2d.SIFT_create()
    for ii, file in enumerate(filelist):
        print("Indexing %s" % (file,))
        im = cv2.cvtColor(cv2.imread("../data/images/" + file), cv2.COLOR_BGR2GRAY)
        # read the descriptors
        kp, desc = sift.detectAndCompute(im, None)
        if desc is None:
            # let the descriptor be 0 for all values
            # note that this is bad and the element should be dropped from the index
            print("WARNING: zero descriptor for %s" % (shortname,))
            continue
        
        # convert to float
        desc = desc.astype(np.float32)
        
        # center and apply PCA transform
        desc = desc - np.mean(desc, axis=1, keepdims=True)
        desc = np.dot(desc, pca_transform)
        
        # get cluster ids
        clabels = kmeans.predict(desc)
        # compute histogram
        descr_hist = np.histogram(clabels, bins=kmeans.n_clusters)[0]
        
        descr_hist = np.reshape(descr_hist, (1, -1))
        # l1 norm
        descr_hist = l1_normalizer.transform(descr_hist)
        
        # take the sqrt (Hellinger kernel)
        descr_hist = np.sqrt(descr_hist)
        
        # l2 norm
        descr_hist = l2_normalizer.transform(descr_hist)
        
        # update the index
        image_descriptors.append(descr_hist)
    print("Indexing complete.")
    return image_descriptors

In [90]:
x_train_desc, y_train_desc = [compute_descriptors(filelist, train_mean, pca_transform, kmeans) 
    for filelist in (x_train, y_train)]

Indexing img_0073149


error: OpenCV(3.4.2) /io/opencv/modules/imgproc/src/color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'CvtHelper'
